### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

### Preprocessing
* http://cs231n.github.io/neural-networks-2/#datapre
* http://www.robots.ox.ac.uk/~vgg/practicals/cnn/#getting-started
* https://www.safaribooksonline.com/library/view/programming-computer-vision/9781449341916/ch01.html
* https://stackoverflow.com/a/10169025/5151861
* augmentation tf+keras http://machinelearningmastery.com/image-augmentation-deep-learning-keras/

#### Augmentation
* rotation: random with angle between 0° and 360° (uniform)
* translation: random with shift between -10 and 10 pixels (uniform)
* rescaling: random with scale factor between 1/1.6 and 1.6 (log-uniform)
* flipping: yes or no (bernoulli)
* shearing: random with angle between -20° and 20° (uniform)
* stretching: random with stretch factor between 1/1.3 and 1.3 (log-uniform)
* whitening
* https://www.tensorflow.org/api_docs/python/tf/image
* https://github.com/aleju/imgaug
* http://augmentor.readthedocs.io/en/master/
* https://github.com/analysiscenter/dataset

* uint8 0 to 255
* uint16 0 to 65535
* uint32 0 to 232
* float -1 to 1 or 0 to 1
* int8 -128 to 127
* int16 -32768 to 32767
* int32-231 to 231 - 1

* Gaussian blurring of images
* Morphology—Counting Objects

Ресайзинг - важно
* http://ufldl.stanford.edu/wiki/index.php/UFLDL_Tutorial

* одну- дву- слойную cnn
* графики ошибки
* 224х224
* 32х32
* в керас
* серые картинки 
* тензор борд
* презентация по аугментации

In [1]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

%matplotlib inline

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('data/data_analys.csv', index_col=0)

### Load train transformed

In [3]:
%%time

dir_train_transformed = 'data/train_transformed'
# img parameters
img_shape = (64, 64)
channels = 3
img_shape_flattened = img_shape[0] * img_shape[1] * channels
img_qty = train_df.shape[0]

# initialize X,y
X = np.empty(shape=(img_qty, img_shape_flattened), dtype=np.int8)
y = np.empty(shape=(img_qty,), dtype=np.uint16)

# read images
for i,f_name in enumerate(os.listdir(dir_train_transformed)):
    if i % 5000 == 0:
        print('{:6d}/{:6d} images loaded'.format(i, img_qty))
    
    img_path = os.path.join(dir_train_transformed, f_name)
    X[i, :] = misc.imread(img_path).flatten('C') # since img is np.ndarray, flatten in row-style
    y[i] = train_df.loc[train_df['image_name'] == f_name, 'target'].iloc[0]

     0/ 22897 images loaded
  5000/ 22897 images loaded
 10000/ 22897 images loaded
 15000/ 22897 images loaded
 20000/ 22897 images loaded
CPU times: user 1min 16s, sys: 3.96 s, total: 1min 20s
Wall time: 1min 28s


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y)

### Find optimal PCA components

In [7]:
%%time
pca = PCA(n_components=5000).fit(X_train)
print('{}'.format(np.cumsum(pca.explained_variance_ratio_)))

[ 0.11819607  0.15731014  0.18937642 ...,  0.94386439  0.94388505
  0.94390567]
CPU times: user 34min, sys: 1min 53s, total: 35min 53s
Wall time: 15min 46s


In [ ]:
round(np.cumsum(pca.explained_variance_ratio_)[-1400], 4)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xticks(np.arange(0, 5000, 500))
plt.axhline(0.9, c='r');

In [ ]:
%%time
pca = PCA(n_components=3600)

X_train_sc = X_train / 255
X_train_pca = pca.fit_transform(X_train_sc)

In [ ]:
%%time
X_test_sc = X_test / 255
X_test_pca = pca.transform(X_test_sc)

In [ ]:
reg = LogisticRegression(random_state=42, C=0.1, max_iter=50, verbose=2, n_jobs=-1, class_weight='balanced')
reg.fit(X_train_pca, y_train)

In [ ]:
accuracy(y_train, reg.predict(X_train_pca))
accuracy(y_test, reg.predict(X_test_pca))

In [ ]:
def accuracy(y, y_pred):
    print('accuracy: {}'.format(accuracy_score(y, y_pred)))

### Making pipeline

In [ ]:
pca = PCA(n_components=3600)
svc = LogisticRegression(random_state=42, C=0.1, max_iter=50, verbose=2, n_jobs=-1, class_weight='balanced')
scaler = StandardScaler()

model = Pipeline((('scaler', scaler), ('dim_reduction', pca), ('classifier', svc)))

In [ ]:
%%time
model.fit(X_train, y_train)

### Load test transformed

In [ ]:
%%time
dir_test_transformed = 'data/test_transformed'
dir_test = 'data/test'
img_size=(64, 64)
check_dir(dir_test_transformed)
for ind, image_name in enumerate(os.listdir(dir_test)):
        if ind % 600 == 0:
            print('{}'.format(ind))
        if not image_name.startswith('.'):
            name = os.path.join(dir_test, image_name)
            image = misc.imread(name)
            if len(image.shape) == 2:
                image = np.stack((image,)*3)
            image = misc.imresize(image, img_size, interp='bicubic')
            #image = norm(image)
            fold_to_save = os.path.join(dir_test_transformed, image_name)
            misc.imsave(fold_to_save, image)

In [ ]:
%%time

X_test_pca = pca.transform(X_test)
y_pred = svc.predict(X_test_pca)

sub = pd.DataFrame(
    data=[row for row in zip(os.listdir(dir_test_transformed), y_pred.astype(int).tolist())],
    columns=['image', 'class']
)
sub.to_csv('lr_baseline.csv', encoding='utf-8', index=False)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
img = misc.imread('data/train/001.ak47/001_0001.jpg')  # this is a PIL image
x = X[]  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(X_test[0], batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_1/MaxPool' (op: 'MaxPool') with input shapes: [?,1,148,32].